# **Data Cleaning and Transformation Project**

## **Aims**

- Change the data types of the height and weight columns
- Clean and transform the 'Value', 'Wage' and 'Release Clause' columns into columns of floats
- Create a 'Joined Year' column and a 'Time at Club' column
- Remove 'star' characters from columns and make these columns numerical
- Remove the newline characters from the 'Hits' column


### **Import csv files into Pandas dataframe** ###

In [24]:
import numpy as np
import pandas as pd


players = pd.read_csv('fifa21 raw data v2.csv',
                      usecols=['Joined', 'Value', 'Wage', 'Release Clause', 'Height', 'Weight', 'W/F', 'SM', 'IR', 'Hits'],
                      dtype={'Hits': 'string'},
                      header=0)

### **Change the data types of the height and weight columns**



#### **Height** ####

In [25]:
import re
# Remove cm from any occurences in the Height column
players['Height'] = players['Height'].str.replace('cm', '')

# Function to check if a height is in feet'inches format and convert it
def convert_height(Height):
    if re.match(r"^\d+'\d+$", Height):  # Matches formats like "5'9"
        feet, inches = map(int, Height.split("'"))
        return feet * 30.48 + inches * 2.54
    else:
        return float(Height)  # Assume it's already in cm

# Apply the function
players['Height_cm'] = players['Height'].apply(convert_height)

# Convert the column to integers
players['Height_cm'] = players['Height_cm'].astype(int)

print(players.dtypes)

Height                    object
Weight                    object
Joined                    object
Value                     object
Wage                      object
Release Clause            object
W/F                       object
SM                        object
IR                        object
Hits              string[python]
Height_cm                  int64
dtype: object


In [26]:
# Replace Height column with Height_cm column
players['Height'] = players['Height_cm']

# Drop the Height_cm column
players = players.drop(columns=['Height_cm'])

print(players.dtypes)

Height                     int64
Weight                    object
Joined                    object
Value                     object
Wage                      object
Release Clause            object
W/F                       object
SM                        object
IR                        object
Hits              string[python]
dtype: object


#### **Weight** ####

In [27]:
# Remove kg from any occurences in the Weight column
players['Weight'] = players['Weight'].str.replace('kg', '')

# Remove lbs from any occurences in the Weight column
players['Weight'] = players['Weight'].str.replace('lbs', '')

# Convert the column to integers
players['Weight'] = players['Weight'].astype(int)

# Function to convert lbs to kg if below threshold (assuming weights under 120 are in kg)
def convert_weight(Weight):
    return Weight * 0.453592 if Weight > 120 else Weight  # Convert only lbs to kg

# Apply function to the column
players['weight_kg'] = players['Weight'].apply(convert_weight)

print(players.dtypes)


Height                     int64
Weight                     int64
Joined                    object
Value                     object
Wage                      object
Release Clause            object
W/F                       object
SM                        object
IR                        object
Hits              string[python]
weight_kg                float64
dtype: object


### **Clean and transform the 'Value', 'Wage' and 'Release Clause' columns into columns of floats**

In [28]:
# Code for removing thousands(K), millions(M) and billions(B) suffixes

def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return x

In [29]:
# Code to remove currency signs and convert string columns to float columns
players['Wage'] = players['Wage'].str[1:].apply(value_to_float).astype(float)
players['Value'] = players['Value'].str[1:].apply(value_to_float).astype(float)
players['Release Clause'] = players['Release Clause'].str[1:].apply(value_to_float).astype(float)

In [30]:
players['Wage']

0        560000.0
1        220000.0
2        125000.0
3        370000.0
4        270000.0
           ...   
18974      1000.0
18975       500.0
18976       500.0
18977      2000.0
18978      1000.0
Name: Wage, Length: 18979, dtype: float64

In [31]:
players['Value']

0        103500000.0
1         63000000.0
2        120000000.0
3        129000000.0
4        132000000.0
            ...     
18974       100000.0
18975       130000.0
18976       120000.0
18977       100000.0
18978       100000.0
Name: Value, Length: 18979, dtype: float64

In [32]:
players['Release Clause']

0        138400000.0
1         75900000.0
2        159400000.0
3        161000000.0
4        166500000.0
            ...     
18974        70000.0
18975       165000.0
18976       131000.0
18977        88000.0
18978        79000.0
Name: Release Clause, Length: 18979, dtype: float64

### **Create a 'Joined Year' column and a 'Time at Club' column**

In [33]:
# Code to create 'Joined Year' and 'Time at Club' columns
players['Joined Year'] = players['Joined'].str[-4:].astype(int)
players['Time at Club'] = 2024 - players['Joined Year']


In [34]:
players['Time at Club']

0        20
1         6
2        10
3         9
4         7
         ..
18974     6
18975     4
18976     5
18977     4
18978     5
Name: Time at Club, Length: 18979, dtype: int64

### **Remove 'star' characters from columns and make these columns numerical**

In [35]:
# Code to remove 'star' characters
players['W/F'] = players['W/F'].str[:-2].astype(int)
players['SM'] = players['SM'].str[:-1].astype(int)
players['IR'] = players['IR'].str[:-2].astype(int)

In [36]:
players['W/F']

0        4
1        4
2        3
3        5
4        5
        ..
18974    2
18975    2
18976    2
18977    3
18978    3
Name: W/F, Length: 18979, dtype: int64

In [37]:
players['SM']

0        4
1        5
2        1
3        4
4        5
        ..
18974    2
18975    2
18976    2
18977    2
18978    2
Name: SM, Length: 18979, dtype: int64

In [38]:
players['IR']

0        5
1        5
2        3
3        4
4        5
        ..
18974    1
18975    1
18976    1
18977    1
18978    1
Name: IR, Length: 18979, dtype: int64

### **Remove the newline characters from the 'Hits' column**

In [39]:
# Code to remove newline characters from 'Hits' column
players['Hits'] = players['Hits'].str.replace('\n', '')

In [40]:
players['Hits']

0         771
1         562
2         150
3         207
4         595
         ... 
18974    <NA>
18975    <NA>
18976    <NA>
18977    <NA>
18978    <NA>
Name: Hits, Length: 18979, dtype: string